In [6]:
import pandas as pd
import openpyxl
import geopandas as gp
import geopy
import os
from pathlib import Path

In [7]:
df = pd.read_excel(Path(os.getcwd(), "Assets", "Data", "locations.xlsx"))
df.head()

,Brand,Name,Address,Open_time,Close_time
0,FEBO,FEBO Reguliersbreestraat Amsterdam,Reguliersbreestraat 38 1017 CN Amsterdam,"{“Maandag”: “11:00”, “Dinsdag”: “11:00”, “Woen...","{“Maandag”: “3:00”, “Dinsdag”: “3:00”, “Woensd..."
1,FEBO,FEBO Hoofddorp,Marktplein 220 2132 CW Hoofddorp,NaN,NaN
2,FEBO,FEBO Diagonaal Almere,De Diagonaal 26 1315 XE Almere,NaN,NaN
3,FEBO,FEBO Almere-Buiten,Noordeinde 176 1334 BR Almere,NaN,NaN
4,FEBO,FEBO Amersfoort,Langestraat 84 3811 AK Amersfoort,NaN,NaN


In [8]:
# Geocoding
geolocator = geopy.geocoders.Nominatim(user_agent="test")
location = geolocator.geocode(df.Address.iloc[0])
print(location.raw)

{'place_id': 144018272, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1719407868, 'lat': '52.3663683', 'lon': '4.8951869', 'class': 'amenity', 'type': 'fast_food', 'place_rank': 30, 'importance': 8.268177309829888e-05, 'addresstype': 'amenity', 'name': 'FEBO', 'display_name': 'FEBO, 38, Reguliersbreestraat, Grachtengordel, Centrum, Amsterdam, Noord-Holland, Nederland, 1017 CN, Nederland', 'boundingbox': ['52.3663183', '52.3664183', '4.8951369', '4.8952369']}


In [ ]:
proj_name = FoodPrisonFinder

def get_coords(row, geo_agent):
    location = geo_agent.geocode(row["Address"]).raw
    row["y"] = location.raw['lat']
    row["x"] = location.raw['long']
    return row

def address_to_geojson(df: pd.DataFrame, save_name: str, in_coord_sys: str = "EPSG:4326", out_coord_sys: str = "EPSG:4326"):
    # Make geo agent and then apply geocode fn
    geo_agent = geopy.geocoders.Nominatim(user_agent = proj_name)
    df = df.apply(lambda x: get_coords(x, geo_agent))
    # Convert to geodataframe and then export in GeoJson format
    gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.x, df.y), crs = in_coord_sys)
    gdf.to_json()
    print("File saved.")


'52.3663683'